In [2]:
import datasets
from compare_summary_bert import split_in_sents, eval_score
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import evaluate
import pandas as pd

In [4]:

cnn_data = datasets.load_dataset("cnn_dailymail", '2.0.0') #,cache_dir='/var/scratch/mca305')
cnn_data_test = cnn_data['test']
cnn_data_validation = cnn_data['validation']
small_DS = datasets.load_from_disk("data_parsed_roberta_edus")
small_DS = cnn_data_validation.filter(lambda x: x['id'] in small_DS['article_id'])
small_DS = small_DS.map(split_in_sents, remove_columns=small_DS.column_names, batched=True)

In [9]:
model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
a = model.encode(small_DS['sentence'][0])
#ds = small_DS.map(lambda x: model.encode(x['sentence']))
#ds.set_format(type='torch', columns=['sentence', 'highlight', 'article_id'])

In [11]:
len(a)

1024

In [ ]:

rouge = evaluate.load('rouge')
eval_methods = {
    'Method': [],
    'Summary': [],
    'Label': [],
    'rouge1': [],
    'rouge2': [],
    'rougeL': [],
    'rougeLsum': []
}

In [ ]:

unique_ids = list(set(small_DS['article_id']))
cnt = 0
for u_id in tqdm(unique_ids):
    current_ds = small_DS.filter(lambda x: x['article_id'] == u_id)

    sents = current_ds['sentence']
    summary = current_ds['highlight'][0]

    # Eval scores BERT
    k_center, stl_k_center, random_e, k_center_trim, summaries = eval_score(sents, current_ds['embeddings'], summary, rouge)

    # k_center
    eval_methods['Method'].append('K Center')
    eval_methods['Summary'].append(summaries[0])
    eval_methods['Label'].append(summary)
    for k in list(k_center.keys()):
        eval_methods[k].append(k_center[k])

    # st_k_center
    eval_methods['Method'].append('Salient STL')
    eval_methods['Summary'].append(summaries[1])
    eval_methods['Label'].append(summary)
    for k in list(stl_k_center.keys()):
        eval_methods[k].append(stl_k_center[k])

    # random
    eval_methods['Method'].append('Random')
    eval_methods['Summary'].append(summaries[2])
    eval_methods['Label'].append(summary)
    for k in list(random_e.keys()):
        eval_methods[k].append(random_e[k])

    # K Center Trimmed
    eval_methods['Method'].append('K Center STL')
    eval_methods['Summary'].append(summaries[3])
    eval_methods['Label'].append(summary)
    for k in list(k_center_trim.keys()):
        eval_methods[k].append(k_center_trim[k])
    cnt += 1

    ## Eval scores ROBERTA
    print(cnt)

#df = pd.DataFrame.from_dict(eval_methods)
#df.to_csv('scoring_roberta_base_1_st.csv')